In [1]:
import numpy as np
import os
import sys
import matplotlib.pyplot as plt

In [2]:
sys.path.append('/Users/fllorente/Dropbox/con_Petar/PYTHON/gp_fusion')


In [10]:
from modules.data_handling import load_data, load_and_normalize_data
from modules.model_training import train_and_predict_single_gp
from modules.model_training import GPModel, to_torch
from modules.fusion_methods import product_fusion
from modules.fusion_methods import compute_neg_log_like


import torch
from tqdm import tqdm

from gpytorch.means import ZeroMean
from gpytorch.kernels import AdditiveStructureKernel, RBFKernel, ScaleKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.metrics import mean_standardized_log_loss




In [70]:
# ------------ Load and normalize data --------- #
dataset_name = "autos"

split=5
X_train, y_train, X_test, y_test, y_std = load_data(dataset_name,split,)

print('input dim: ', X_train.shape[1])
print("training size: ", len(y_train))
print("test size: ", len(y_test))

input dim:  24
training size:  143
test size:  16


In [5]:
# check that the entire dataset has zero mean (I didn't do this, but Trevor :-))
print(np.mean(np.vstack([X_train,X_test]),axis=0))
print(np.mean(np.concatenate([y_train,y_test]),axis=0))

# I did normalize the intputs and outputs to have unit variance
print(np.std(np.vstack([X_train,X_test]),axis=0))
print(np.std(np.concatenate([y_train,y_test]),axis=0))

[ 1.63930854e-05  2.37138165e-06  1.15270825e-05  1.29099445e-06
  3.51763235e-06 -8.72081599e-06  6.12994837e-06  6.03475016e-06
 -5.31113006e-06 -4.72182186e-06  1.41440516e-05 -8.71014121e-06
  4.29387139e-07  1.30303597e-06  6.78165922e-06  8.92713126e-06
 -1.77088659e-05 -1.33294018e-05  3.28627486e-06  7.92884062e-06
  1.14154070e-05 -5.14751347e-06  1.80051111e-06 -8.41591323e-06]
1.0397028718051562e-07
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
1.0


In [71]:
# Fit a GP with ARD-RBF kernel as the baseline

test_preds, _ = train_and_predict_single_gp(X_train,y_train,X_test,X_test,
                            kappa=2,lambdaa=2,
                            mean=ZeroMean(),  # we don't want to use the mean of y_train as prior mean (using ConstantMean())
                            lr=0.1,
                            training_iter=100,
                            initialiaze_hyper=True  # if false, kappa and lambdaa are not used for initializing the hyperparameter
                            )

# compute the negative log-likelihood (log-loss) on test data:
print(mean_standardized_log_loss(test_preds, torch.from_numpy(y_test)))

# equivalently:
print(compute_neg_log_like(test_preds.mean.numpy(),
                           np.sqrt(test_preds.variance).numpy(),
                           y_test))

tensor(-0.0174, dtype=torch.float64)
[-0.01738962]


In [9]:
# Let us do the check that we get the same results when we train and predict using the functions:
# train_expert and predict_with_expert
from modules.model_training import train_expert, predict_with_expert


model,likelihood = train_expert(X_train,y_train,
                                kappa=2,lambdaa=2,
                                mean=ZeroMean(),
                                lr=0.1,
                                training_iter=500,
                                initialize_hyper=False,
                                )

mean_preds,std_preds,_ = predict_with_expert(model,likelihood,X_test)

print(compute_neg_log_like(mean_preds,std_preds,y_test))

[0.39663261]


Let's average results

In [111]:

nlpd = []
rmse = []
for i in tqdm(range(10)):
    try:
        X_train, y_train, X_test, y_test, y_std = load_data(dataset_name,i)

        # # With load_and_normalize_data fun the data is normalized using the training data only
        # X_train, y_train, X_test, y_test = load_and_normalize_data(dataset_name,i,
        #                                                         normalize_y=True,  
        #                                                         normalize_x_method="max-min")

        test_preds, _ = train_and_predict_single_gp(X_train,y_train,X_test,X_test,
                                kappa=2,lambdaa=2,
                                mean=ZeroMean(),  # we don't want to use the mean of y_train as prior mean
                                lr=0.1,
                                training_iter=100,
                                initialiaze_hyper=False, # if false, kappa and lambdaa don't matter!
                                )
        nlpd_now = compute_neg_log_like(test_preds.mean,np.sqrt(test_preds.variance),y_test)

        rmse_now = np.sqrt(np.mean((test_preds.mean.numpy() - y_test)**2))

        nlpd.append(nlpd_now.squeeze())
        rmse.append(rmse_now)
    except:
        print("There was an error during hyperparameter learning.")


nlpd = np.array(nlpd)
rmse = np.array(rmse)

100%|██████████| 10/10 [00:01<00:00,  6.10it/s]


In [112]:
nlpd

array([ 0.50446794,  0.72029356,  0.05992493,  0.32965062,  0.11765238,
       -0.02974258,  0.13023134,  0.1442318 ,  0.1408854 ,  0.37034965])

In [113]:
nlpd.mean(),nlpd.std()

(0.248794504080366, 0.2181929104889972)

In [114]:
rmse.mean(), rmse.std()

(0.35070544138449566, 0.13871238501254016)

Let's try using an additive kernel

In [107]:
'''
AdditiveStructureKernel computes d one-dimensional kernels (using the supplied base_kernel), 
and then adds the component kernels together.
"AdditiveStructureKernel is deprecated, and will be removed in GPyTorch 2.0. "
'Please refer to the "Kernels with Additive or Product Structure" tutorial '
"in the GPyTorch docs for how to implement GPs with additive structure."
'''
kernel = AdditiveStructureKernel(base_kernel=ScaleKernel(RBFKernel()), num_dims=X_train.shape[1])

nlpd = []
rmse = []
for i in tqdm(range(10)):
    # try:
        X_train, y_train, X_test, y_test, y_std = load_data(dataset_name,i)

        # # With load_and_normalize_data fun the data is normalized using the training data only
        # X_train, y_train, X_test, y_test = load_and_normalize_data(dataset_name,i,
        #                                                         normalize_y=True,  
        #                                                         normalize_x_method="max-min")

        test_preds, _ = train_and_predict_single_gp(X_train,y_train,X_test,X_test,
                                kappa=2,lambdaa=2,
                                mean=ZeroMean(),  # we don't want to use the mean of y_train as prior mean
                                kernel=kernel,
                                lr=0.1,
                                training_iter=100,
                                initialiaze_hyper=False, # if False, kappa and lambdaa are not used for initializing the hyperparameters; we just use the default values.
                                )
        nlpd_now = compute_neg_log_like(test_preds.mean,np.sqrt(test_preds.variance),y_test)

        rmse_now = np.sqrt(np.mean((test_preds.mean.detach().numpy() - y_test)**2))

        nlpd.append(nlpd_now.squeeze())
        rmse.append(rmse_now)
    # except:
    #     print("There was an error during hyperparameter learning.")


nlpd = np.array(nlpd)
rmse = np.array(rmse)

100%|██████████| 10/10 [00:04<00:00,  2.37it/s]


In [110]:
nlpd

array([0.34137439, 0.6400602 , 0.37289921, 0.16129038, 0.13417607,
       0.04016093, 0.02746184, 0.02168028, 0.0959663 , 0.19244998])

In [108]:
nlpd.mean(),nlpd.std()

(0.20275195698146095, 0.18635589889484833)

In [109]:
rmse.mean(), rmse.std()

(0.3100164964961764, 0.08046684862617143)

In [22]:
%%script true
kernel = ScaleKernel(AdditiveStructureKernel(base_kernel=RBFKernel(), num_dims=X_train.shape[1]))
model = GPModel(to_torch(X_train),
                to_torch(y_train),
                GaussianLikelihood(),
                kernel=kernel,mean=ZeroMean())
for name, param in model.named_parameters():
    print(f'Parameter name: {name:42} value = {param.item():1.2f}')

## Stack GPs built on 1d projections:

We'll use the axis-aligned 1d projections for the moment, since I think it's the one
"closer" to using the additive kernel of one-dimensional kernels

In [103]:
nlpd = []
rmse = []

proj = "axis"

for split in tqdm(range(10)):
        X_train, y_train, X_test, y_test, y_std = load_data(dataset_name,split)

        mean_experts = []
        std_experts = []
        for d in range(X_train.shape[1]):
            if proj == "random":
                  P_proj = np.random.rand(X_train.shape[1],1)
            elif proj == "axis":
                  index = d
                  P_proj = np.array([1.0 if i == index else 0.0 for i in range(X_train.shape[1])])
                  P_proj = P_proj.reshape(-1,1)

            test_preds, _ = train_and_predict_single_gp(
                                    np.matmul(X_train,P_proj),
                                    y_train,
                                    np.matmul(X_test,P_proj),
                                    np.matmul(X_test,P_proj),
                                    mean=ZeroMean(),  # we don't want to use the mean of y_train as prior mean
                                    lr=0.1,
                                    training_iter=100,
                                    initialiaze_hyper=False, # if False, kappa and lambdaa are not used for initializing the hyperparameters; we just use the default values.
                                    )
            mean_experts.append(test_preds.mean.numpy())
            std_experts.append(np.sqrt(test_preds.variance.numpy()))

        mean_experts = np.stack(mean_experts,axis=-1)
        std_experts = np.stack(std_experts,axis=-1)


        # Fuse one-dimensional experts' predictions
        mean_fused, std_fused, _ = product_fusion(mean_experts,std_experts,
                                                  weighting="uniform",
                                                #   weighting="no-weights",
                                                #   normalize=False,
                                                  method = "gPoE")
        
        nlpd_now = compute_neg_log_like(mean_fused,std_fused,y_test)
        nlpd.append(nlpd_now.squeeze())

        rmse_now = np.sqrt(np.mean( (mean_fused-y_test)**2 ))
        rmse.append(rmse_now)
        
nlpd = np.array(nlpd)
rmse = np.array(rmse)


100%|██████████| 10/10 [00:34<00:00,  3.42s/it]


In [104]:
nlpd.mean(), nlpd.std()

(0.8012721086485826, 0.2046680391316493)

In [105]:
rmse.mean(), rmse.std()

(1.1113879047936768, 0.23457900304804294)

## I want to try some of the models and training setups of Delbridge's repository

In [29]:
sys.path.append('/Users/fllorente/Dropbox/con_Petar/PYTHON/Randomly-Projected-Additive-GPs')


In [6]:
from fitting.optimizing import train_to_convergence

In [72]:
from gpytorch.mlls import ExactMarginalLogLikelihood

likelihood = GaussianLikelihood()
model = GPModel(to_torch(X_train),to_torch(y_train),likelihood)

print(model.covar_module.base_kernel.lengthscale)
print(likelihood.noise_covar.noise)


mll = ExactMarginalLogLikelihood(model.likelihood, model)
max_iter = train_to_convergence(model, 
                     to_torch(X_train), 
                     to_torch(y_train),
                     optimizer=torch.optim.Adam, objective=mll, 
                     max_iter=100, print_freq=1, verbose=0,lr = 0.1)
max_iter

tensor([[0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931,
         0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931,
         0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931]],
       grad_fn=<SoftplusBackward0>)
tensor([0.6932], grad_fn=<AddBackward0>)


/Users/fllorente/Dropbox/con_Petar/PYTHON/Randomly-Projected-Additive-GPs/fitting/optimizing.py:81: RuntimeWarning: Mean of empty slice.
  ma[i] = losses[i-patience+1:i+1].mean()
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


100

In [73]:
print(model.covar_module.base_kernel.lengthscale)
print(likelihood.noise_covar.noise)

tensor([[ 6.7088,  6.1273,  4.9083,  7.5706,  7.0607,  6.5998,  6.1216,  4.9640,
          6.1993,  5.5571,  6.3582,  4.8340,  2.3613,  4.9477,  6.8069,  7.2692,
          6.6578,  8.0215,  6.3105, 10.6027,  5.4907,  6.9049,  5.4511,  7.0181]],
       grad_fn=<SoftplusBackward0>)
tensor([0.0450], grad_fn=<AddBackward0>)


In [74]:
model.eval()
test_preds = likelihood(model(to_torch(X_test)))

print(mean_standardized_log_loss(test_preds,to_torch(y_test)))

tensor(-0.0292, grad_fn=<MeanBackward1>)


I want to try now the function create_exact_gp

In [75]:
from training_routines import train_exact_gp

In [99]:
model_args = {'ard': False, 'ski': False, 'grid_size': None, 'kernel_type': 'RBF', 
              'init_lengthscale_range': (0.1, 1.0), 'keops': False,
              'noise_prior': False, 'init_noise_range': (0.1,1)
              }
train_args = {'lr': 0.1, 'max_iter': 100, 'verbose': 0, 'patience': 20,
              'conv_tol': 1e-4, 'check_conv': True, 'smooth': True,
              'batch_size': None, 'checkpoint': False, 'print_freq': 1,
              'random_restarts': 5, 'optimizer': 'adam',
              }

model_metrics, pred_mean, model = train_exact_gp(trainX = to_torch(X_train), 
                                                 trainY = to_torch(y_train), 
                                                 testX = to_torch(X_test), 
                                                 testY = to_torch(y_test), 
                                                 kind = 'full', 
                                                 model_kwargs = model_args, 
                                                 train_kwargs = train_args,
                                                 )

/Users/fllorente/Dropbox/con_Petar/PYTHON/Randomly-Projected-Additive-GPs/fitting/optimizing.py:81: RuntimeWarning: Mean of empty slice.
  ma[i] = losses[i-patience+1:i+1].mean()
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gpytorch/models/exact_gp.py:284: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(


In [100]:
model_metrics["prior_train_nmll"], model_metrics["train_nll"]

(0.40398284792900085, -0.1583402454853058)

In [102]:
mean_standardized_log_loss(model.likelihood(model(to_torch(X_train))),
                           to_torch(y_train))

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gpytorch/models/exact_gp.py:284: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(


tensor(-0.1796, grad_fn=<MeanBackward1>)

In [107]:
print(model_metrics["test_nll"]) # creo que esta mal calculado...
'''
esta usando mll = GaussianLogMarginalLikelihood para calcular este valor...
por lo que estaria calculando la log-pdf de la Gaussian multivariate, en lugar de la
media de las log-pdf de las Gaussianas univariantes.
'''

mll = ExactMarginalLogLikelihood(model.likelihood, model)
mll.eval()
aux =model(to_torch(X_test))
print(-mll(aux,to_torch(y_test)).detach().numpy())


# deberia ser igual a esto pero no...
aux = model.likelihood(model(to_torch(X_test)))

from scipy.stats import multivariate_normal

print(
    -multivariate_normal.logpdf(y_test, 
                           aux.mean.detach().numpy(),
                           aux.covariance_matrix.detach().numpy(),
                           )
)

# este valor coincide con
print(-aux.log_prob(to_torch(y_test)).detach().numpy())

0.02306675910949707
0.02306676
0.36906733155155
0.36906815


In [112]:
with torch.no_grad():
    print(mean_standardized_log_loss(model.likelihood(model(to_torch(X_test))),
                           to_torch(y_test)))

tensor(0.0133)


In [80]:
pred_mean

tensor([-0.9929,  0.9791,  0.0551, -0.3986, -0.5403, -0.3692, -0.3082, -0.4199,
        -0.9787,  0.4410, -0.4687, -1.3667,  1.5264,  1.0115, -0.6052, -0.6271])

In [81]:
model.likelihood(model(to_torch(X_test))).mean

tensor([-0.9929,  0.9791,  0.0551, -0.3986, -0.5403, -0.3692, -0.3082, -0.4199,
        -0.9787,  0.4410, -0.4687, -1.3667,  1.5264,  1.0115, -0.6052, -0.6271],
       grad_fn=<ViewBackward0>)